# Offline Policy Evaluation w/ minibatch

Testing offline policy evalation with zone-agnostic behavior policy model

In [1]:
import numpy as np
import sys
import matplotlib.pyplot as plt
import pandas as pd
import pickle
sys.path.insert(0, "../")
import torch
from ppo import PPO
import glob
from tqdm.notebook import tqdm

In [2]:
log_data = pd.read_csv("../data/rule_based_log_data/0_cleaned_log.csv")
with open("../data/rule_based_log_data/action_probs_all_data.pkl", "rb") as f:
    behavior_model = pickle.load(f)

## 30 Day MiniBatch

In [3]:
num_ts_per_day = 4 * 24
num_days = 30
ts_end = num_ts_per_day * num_days
zones = log_data["zone"].unique()
# zone = "Perimeter_bot_ZN_1"
# log_data = log_data[log_data["zone"] == zone].sort_values(by=["timestep"])
# log_data = log_data[:ts_end]

## Loading all Policies

In [4]:
policy_list = sorted(list(glob.glob(f"../policy_library/**.pth")))

### 1. Inverse Probability Weighting

In [ ]:
from ope.iw import InverseProbabilityWeighting
policy_scores = {}
num_invalid_policies = 0
for zone in zones[:1]:
    print(zone)
    ope_data = log_data[log_data["zone"] == zone].sort_values(by=["timestep"])
    ope_data = log_data[:ts_end]
    ipw = InverseProbabilityWeighting(ope_data)
    for policy in tqdm(policy_list):
        agent = PPO(6, 1, 0.003, 0.0005, 1, 10, 0.2,
                    has_continuous_action_space=True, action_std_init=0.2, 
                    device=torch.device('cpu'), diverse_policies=list(),
                    diverse_weight=0, diverse_increase=True)
        agent.load(policy)
        agent.policy_evaluation = True
        agent.policy_old.set_action_std(0.1)
        try:
            score = ipw.evaluate_policy(agent.select_action, behavior_model, score="mean")
        except ValueError:
            print(num_invalid_policies+1, policy, " not working")
            num_invalid_policies += 1
            continue
        if policy not in policy_scores:
            policy_scores[policy] = {}
        if zone not in policy_scores[policy]:
            policy_scores[policy][zone] = score
print(num_invalid_policies+1)

Perimeter_top_ZN_3


  0%|          | 0/800 [00:00<?, ?it/s]

1 ../policy_library/100_0_1e-1.pth  not working
2 ../policy_library/100_0_1e-1_blind.pth  not working
3 ../policy_library/100_0_1e0.pth  not working
4 ../policy_library/100_0_1e0_blind.pth  not working
5 ../policy_library/100_0_1e1.pth  not working
6 ../policy_library/100_0_1e1_blind.pth  not working
7 ../policy_library/100_1_1e-1_blind.pth  not working
8 ../policy_library/100_1_1e0_blind.pth  not working


In [18]:
policy_scores

{'../policy_library/100_0.pth': {'Perimeter_top_ZN_3': 98838885.74976222}}